In [168]:
#tworzy worek słów z pliku
#file_name - nazwa pliku z której utworzyc worek słów; with_stop_words - True/False mówiący czy usuwać stop wordsy czy nie
def create_bag_of_words_from_file(file_name, with_stop_words=False):
#struktura worka: indeks to indeks, elementem jest lista ze słowem i liczebnością
    bag = {}
    stop_words = []
#tablica znaków interpunkcyjnych do pozbycia się z wyrazów
#TODO: jakieś jeszcze inne znaki?
    punc = '''…!()-[]{};:'"\,<>./?@#$%^&*_~«—'''
#ewentualne przypisanie stopwordsów
    if not with_stop_words:
        stop_words = get_stop_words_from_file('stopWords.txt')
        
    with open(file_name,'r', encoding='utf-8') as file:
#autor
        author = file.readline()
        author = author.replace("\n","")
        bag[0] = author

#przeskoczenie przez tytuł i ewentualny ISBN
        for i in range(3):
            file.readline()
        position_before = file.tell()
        if not file.readline().startswith('ISBN'):
            file.seek(position_before)
            

        for line in file:
#jak dotrze do stopki-kończ
            for word in line.split():
                if word == "-----":
                    return bag
#usuwanie znaków interpunkcyjnych
                for ele in word:
                    if ele in punc:
                        word = word.replace(ele, "")
                word = word.lower()
#jeśli jest stop wordsem- nie uwzględniamy (jeśli w funkcji było True, to 'stop_words' jest puste, czyli nie wejdzie do ifa)    
                if word in stop_words:
                    continue
#dodawanie nowego słowa do worka, jeśli jeszcze go nie ma
                if word not in bag and (len(word) != 0):
                    bag[word] = 1
#inkrementacja liczności słowa, jeśli już istnieje w worku
                elif len(word) != 0:
                    bag[word] += 1
    return bag

#funkcja pobiera stop wordsy z pliku i zwraca je jako listę
def get_stop_words_from_file(file_name):
    stops = []
    borders = ["\\n", "'"]
    with open(file_name, encoding='utf-8') as f:
        for line in f:
            single_line = repr(line)
            for char in borders:
                single_line = single_line.replace(char,"")
            if not(single_line == ""):
                stops.append(single_line)
    return stops

In [169]:
#Do testów TODO: usunąć
b = create_bag_of_words_from_file('panTadeusz.txt')
for i in (b.keys()):
    print("{0} : {1}".format(i, b[i]))

0 : Adam Mickiewicz
księga : 15
pierwsza : 4
gospodarstwo : 3
powrót : 3
panicza : 2
spotkanie : 5
pierwsze : 12
pokoiku : 3
drugie : 8
stołu : 21
ważna : 3
sędziego : 38
nauka : 3
grzeczności : 4
podkomorzego : 10
uwagi : 4
polityczne : 2
modami : 1
początek : 4
sporu : 9
kusego : 10
sokoła : 9
żale : 3
wojskiego : 21
ostatni : 21
woźny : 28
trybunału : 2
rzut : 2
oka : 10
ówczesny : 1
stan : 3
polityczny : 2
litwy : 7
europy : 2
litwo : 1
ojczyzno : 1
jesteś : 22
zdrowie : 18
cenić : 2
dowie : 5
stracił : 3
piękność : 4
twą : 6
całej : 20
ozdobie : 2
widzę : 12
opisuję : 1
tęsknię : 1
panno : 3
święta : 4
jasnej : 2
bronisz : 1
częstochowy : 3
ostrej : 2
świecisz : 1
bramie : 3
gród : 1
zamkowy : 1
nowogródzki : 1
ochraniasz : 1
wiernym : 1
ludem : 4
dziecko : 11
zdrowia : 4
powróciłaś : 1
cudem : 5
płaczącej : 1
matki : 4
twoją : 1
opiekę : 3
ofiarowany : 1
martwą : 1
podniosłem : 1
powiekę : 1
zaraz : 44
mogłem : 6
pieszo : 2
twych : 3
świątyń : 1
progu : 12
iść : 10
wrócone : 1
ży